# 🔄 Preprocessing Data: Cleaning & Preparation 🛠️  
Setelah berhasil melakukan **scraping** data dari berita, langkah selanjutnya adalah **preprocessing**.  
Tahap ini bertujuan untuk membersihkan dan menyiapkan data agar siap untuk analisis lebih lanjut.  

---

## 🔧 **Libraries Used**

In [46]:
import pandas as pd
import re
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from collections import Counter

## 📥 **Import Data CSV**

In [47]:
# Import data
file_path = "../data/hasil_scrap_berita.xlsx"
df = pd.read_excel(file_path)
df

,NO,SUMBER,PENERBIT,LINK,JUDUL,content,tags,date,status
0,2,economic,Detik,https://finance.detik.com/berita-ekonomi-bisni...,Dirjen Pajak Ungkap Kabar Terbaru soal Perbaik...,Direktur Jenderal (Dirjen) Pajak Suryo Utomo m...,"coretax,dirjen pajak,administrasi perpajakan,f...",Tidak ada tanggal,success
1,3,economic,Detik,https://finance.detik.com/berita-ekonomi-bisni...,DJP Klaim Aplikasi Coretax Mulai Stabil Digunakan,Direktorat Jenderal Pajak (DJP) Kementerian Ke...,"coretax,djp,aplikasi perpajakan,pembaruan,ppnb...",25 Maret 2025,success
2,5,local news,DDTC,https://news.ddtc.co.id/berita/nasional/181042...,"Latensi Turun, DJP Klaim Coretax System Sudah ...","Ilustrasi. JAKARTA, DDTCNews - Ditjen Pajak (D...","administrasi pajak, coretax, coretax system, DJP",30 April 2025,success
3,6,economic,Detik,https://finance.detik.com/berita-ekonomi-bisni...,Awas Penipuan Catut Nama Coretax! Begini Modus...,Ditjen Pajak (DJP) Kementerian Keuangan (Kemen...,"penipuan,coretax,ditjen pajak,catut nama coret...",Tidak ada tanggal,success
4,7,local news,Tempo,https://www.tempo.co/ekonomi/hari-ini-direktor...,Hari Ini Direktorat Jenderal Pajak Rapat Tertu...,Scroll ke bawah untuk membaca berita Baca be...,Tidak ada tags,1 Januari 2025,success
...,...,...,...,...,...,...,...,...,...
149,155,local news,CNBC,https://www.cnbcindonesia.com/news/20250114071...,Keluhan Soal Coretax Terus Muncul: Sistem Tak ...,"Jakarta, CNBC Indonesia - Kalangan pengusaha d...","coretax, pajak, djp",1 Januari 2025,success
150,156,local news,Viva,https://www.viva.co.id/bisnis/1809316-coretax-...,"Coretax Bermasalah di Awal Tahun, Misbakhun Te...","Jakarta, VIVA – Meskipun terjadi defisit pada ...","misbakhun, coretax, apbn, golkar, pajak",22 Maret 2025,success
151,157,economic,Tempo,https://www.tempo.co/ekonomi/luhut-sebut-siste...,"Luhut Sebut Sistem Coretax akan Membaik, Apa i...",Scroll ke bawah untuk membaca berita Baca be...,Tidak ada tags,17 Januari 2025,success
152,159,economic,Oke Zone,https://economy.okezone.com/read/2025/04/24/32...,Sri Mulyani Pamer Setoran Pajak Naik Jadi Rp13...,JAKARTA - Menteri Keuangan Sri Mulyani Indrawa...,Tidak ada tags,24 April 2025,success


In [48]:
#Hanya akan menampilkan kolom berikut
df.loc[:,["SUMBER","PENERBIT","JUDUL", "content", "tags", "date"]]

,SUMBER,PENERBIT,JUDUL,content,tags,date
0,economic,Detik,Dirjen Pajak Ungkap Kabar Terbaru soal Perbaik...,Direktur Jenderal (Dirjen) Pajak Suryo Utomo m...,"coretax,dirjen pajak,administrasi perpajakan,f...",Tidak ada tanggal
1,economic,Detik,DJP Klaim Aplikasi Coretax Mulai Stabil Digunakan,Direktorat Jenderal Pajak (DJP) Kementerian Ke...,"coretax,djp,aplikasi perpajakan,pembaruan,ppnb...",25 Maret 2025
2,local news,DDTC,"Latensi Turun, DJP Klaim Coretax System Sudah ...","Ilustrasi. JAKARTA, DDTCNews - Ditjen Pajak (D...","administrasi pajak, coretax, coretax system, DJP",30 April 2025
3,economic,Detik,Awas Penipuan Catut Nama Coretax! Begini Modus...,Ditjen Pajak (DJP) Kementerian Keuangan (Kemen...,"penipuan,coretax,ditjen pajak,catut nama coret...",Tidak ada tanggal
4,local news,Tempo,Hari Ini Direktorat Jenderal Pajak Rapat Tertu...,Scroll ke bawah untuk membaca berita Baca be...,Tidak ada tags,1 Januari 2025
...,...,...,...,...,...,...
149,local news,CNBC,Keluhan Soal Coretax Terus Muncul: Sistem Tak ...,"Jakarta, CNBC Indonesia - Kalangan pengusaha d...","coretax, pajak, djp",1 Januari 2025
150,local news,Viva,"Coretax Bermasalah di Awal Tahun, Misbakhun Te...","Jakarta, VIVA – Meskipun terjadi defisit pada ...","misbakhun, coretax, apbn, golkar, pajak",22 Maret 2025
151,economic,Tempo,"Luhut Sebut Sistem Coretax akan Membaik, Apa i...",Scroll ke bawah untuk membaca berita Baca be...,Tidak ada tags,17 Januari 2025
152,economic,Oke Zone,Sri Mulyani Pamer Setoran Pajak Naik Jadi Rp13...,JAKARTA - Menteri Keuangan Sri Mulyani Indrawa...,Tidak ada tags,24 April 2025


## 📌 **Tahapan Preprocessing**

### ✅ **1. Handling Missing Values** → Menghapus atau mengisi data yang kosong  

In [54]:
print(df.isnull().sum())

NO          0
SUMBER      0
PENERBIT    0
LINK        0
JUDUL       0
content     0
tags        0
date        0
status      0
dtype: int64


Data tidak ada yang kosong

### ✅ **2. Lowercasing** → Mengubah teks menjadi huruf kecil untuk konsistensi

In [55]:
# Ubah ke lowercase hanya pada kolom teks
df['JUDUL'] = df['JUDUL'].str.lower()
df['tags'] = df['tags'].str.lower()
df['content'] = df['content'].str.lower()

In [56]:
df.head()

,NO,SUMBER,PENERBIT,LINK,JUDUL,content,tags,date,status
0,2,economic,Detik,https://finance.detik.com/berita-ekonomi-bisni...,dirjen pajak ungkap kabar terbaru soal perbaik...,direktur jenderal (dirjen) pajak suryo utomo m...,"coretax,dirjen pajak,administrasi perpajakan,f...",Tidak ada tanggal,success
1,3,economic,Detik,https://finance.detik.com/berita-ekonomi-bisni...,djp klaim aplikasi coretax mulai stabil digunakan,direktorat jenderal pajak (djp) kementerian ke...,"coretax,djp,aplikasi perpajakan,pembaruan,ppnb...",25 Maret 2025,success
2,5,local news,DDTC,https://news.ddtc.co.id/berita/nasional/181042...,"latensi turun, djp klaim coretax system sudah ...","ilustrasi. jakarta, ddtcnews - ditjen pajak (d...","administrasi pajak, coretax, coretax system, djp",30 April 2025,success
3,6,economic,Detik,https://finance.detik.com/berita-ekonomi-bisni...,awas penipuan catut nama coretax! begini modus...,ditjen pajak (djp) kementerian keuangan (kemen...,"penipuan,coretax,ditjen pajak,catut nama coret...",Tidak ada tanggal,success
4,7,local news,Tempo,https://www.tempo.co/ekonomi/hari-ini-direktor...,hari ini direktorat jenderal pajak rapat tertu...,scroll ke bawah untuk membaca berita baca be...,tidak ada tags,1 Januari 2025,success


### ✅ **3. Special Characters Removal** → menghapus kata spesial pada berita

In [57]:
def clean_text(text):
    # text = re.sub(r'^[a-zA-Z\s,]+[-\s–]+', '', text)
    text = re.sub(r'\([^)]*\)', '', text) #menghapus teks dalam kurung
    text = re.sub(r'http[s]?://\S+', '', text) #menghapus url
    text = re.sub(r'^[^—-]+[-—]\s*', '', text) ## Menghapus kata penerbit, kota di awal
    text = re.sub(r'^\d{1,2}\s\w+\s\d{4}\s*[-:]\s*', '', text) #menghapus tanggal
    text = re.sub(r'&\w+;', ' ', text)       # karakter html, Contoh: &nbsp; &quot;
    text = re.sub(r'&#\d+;', ' ', text)      # karalter html, Contoh: &#x27;
    text = re.sub(r'\d+', '', text)          # Menghapus angka
    return text

# Terapkan pada kolom 'content'
df['clean_text'] = df['content'].apply(clean_text)


In [58]:
df.loc[:,["content", "clean_text"]]

,content,clean_text
0,direktur jenderal (dirjen) pajak suryo utomo m...,direktur jenderal pajak suryo utomo mengungka...
1,direktorat jenderal pajak (djp) kementerian ke...,"rata berada di bawah , detik, dengan performa ..."
2,"ilustrasi. jakarta, ddtcnews - ditjen pajak (d...",ditjen pajak menilai performa coretax adminis...
3,ditjen pajak (djp) kementerian keuangan (kemen...,akhir ini marak penipuan yang mengatasnamakan ...
4,scroll ke bawah untuk membaca berita baca be...,direktur jenderal atau dirjen pajak kementeria...
...,...,...
149,"jakarta, cnbc indonesia - kalangan pengusaha d...",kalangan pengusaha dan pakar pajak memberikan ...
150,"jakarta, viva – meskipun terjadi defisit pada ...","bulan berikutnya. makanya, misbakhun meminta p..."
151,scroll ke bawah untuk membaca berita baca be...,masih bermasalah di pekan kedua setelah diimpl...
152,jakarta - menteri keuangan sri mulyani indrawa...,menteri keuangan sri mulyani indrawati mengung...


### ✅ **4. Punctuations removal** → Menghapus tanda baca dan simbol yang tidak relevan.

In [59]:
def clean_special_chars(text):
    # Hilangkan emoji dan simbol unicode lainnya
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)  
    
    # Hilangkan tanda baca dan simbol matematika
    text = re.sub(r'[^\w\s]', ' ', text)

    # Hilangkan extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Terapkan
df['content_clean'] = df['clean_text'].apply(clean_special_chars)


In [60]:
df.loc[:,["clean_text", "content_clean"]]

,clean_text,content_clean
0,direktur jenderal pajak suryo utomo mengungka...,direktur jenderal pajak suryo utomo mengungkap...
1,"rata berada di bawah , detik, dengan performa ...",rata berada di bawah detik dengan performa ter...
2,ditjen pajak menilai performa coretax adminis...,ditjen pajak menilai performa coretax administ...
3,akhir ini marak penipuan yang mengatasnamakan ...,akhir ini marak penipuan yang mengatasnamakan ...
4,direktur jenderal atau dirjen pajak kementeria...,direktur jenderal atau dirjen pajak kementeria...
...,...,...
149,kalangan pengusaha dan pakar pajak memberikan ...,kalangan pengusaha dan pakar pajak memberikan ...
150,"bulan berikutnya. makanya, misbakhun meminta p...",bulan berikutnya makanya misbakhun meminta par...
151,masih bermasalah di pekan kedua setelah diimpl...,masih bermasalah di pekan kedua setelah diimpl...
152,menteri keuangan sri mulyani indrawati mengung...,menteri keuangan sri mulyani indrawati mengung...


### ✅ **5. Tokenization** → Memisahkan teks menjadi kata-kata atau unit kecil lainnya.

In [61]:
def tokenize(text):
    return re.findall(r'\b\w+\b', text.lower())

# Apply
df['tokens'] = df['content_clean'].apply(tokenize)

In [62]:
# Cek hasil tokenisasi
df[["content_clean", "tokens"]].head()

,content_clean,tokens
0,direktur jenderal pajak suryo utomo mengungkap...,"[direktur, jenderal, pajak, suryo, utomo, meng..."
1,rata berada di bawah detik dengan performa ter...,"[rata, berada, di, bawah, detik, dengan, perfo..."
2,ditjen pajak menilai performa coretax administ...,"[ditjen, pajak, menilai, performa, coretax, ad..."
3,akhir ini marak penipuan yang mengatasnamakan ...,"[akhir, ini, marak, penipuan, yang, mengatasna..."
4,direktur jenderal atau dirjen pajak kementeria...,"[direktur, jenderal, atau, dirjen, pajak, keme..."


### ✅ **6. Stopword Removal** → Menghapus kata-kata umum yang tidak bermakna dalam analisis

In [63]:
stop_factory = StopWordRemoverFactory()
stop_words = set(stop_factory.get_stop_words())

filtered_stop_words = stop_words.copy()
new_stopwords = ["ada", "lalu", "guna", "klik", "sini", "beri", "bagai", "atas", "bulan", "sama", "kini"] # Bisa menambahkan stopword lain sesuai kebutuhan

def remove_stopwords(tokens):
    cleaned_tokens = [
        token.strip() for token in tokens  # Hapus spasi atau karakter aneh
        if len(token) > 3  # Hapus kata tidak penting
    ]
    # Menghapus stopwords dari token
    return [t for t in cleaned_tokens if t.lower() not in filtered_stop_words]

# Terapkan ke DataFrame
df['stopword_removal'] = df['tokens'].apply(remove_stopwords)

In [64]:
# Cek hasil 
df[["tokens", "stopword_removal"]].head()

,tokens,stopword_removal
0,"[direktur, jenderal, pajak, suryo, utomo, meng...","[direktur, jenderal, pajak, suryo, utomo, meng..."
1,"[rata, berada, di, bawah, detik, dengan, perfo...","[rata, berada, bawah, detik, performa, terbaik..."
2,"[ditjen, pajak, menilai, performa, coretax, ad...","[ditjen, pajak, menilai, performa, coretax, ad..."
3,"[akhir, ini, marak, penipuan, yang, mengatasna...","[akhir, marak, penipuan, mengatasnamakan, impl..."
4,"[direktur, jenderal, atau, dirjen, pajak, keme...","[direktur, jenderal, dirjen, pajak, kementeria..."


### ✅ **7.  Stemming** → Mengubah kata menjadi bentuk dasarnya

In [65]:
# Membuat objek stemmer
stem_factory = StemmerFactory()
stemmer = stem_factory.create_stemmer()

# Fungsi untuk melakukan stemming pada tokens
def stem_tokens(tokens):
    return [stemmer.stem(token) for token in tokens]

# Terapkan stemming ke DataFrame
df['stemming_output'] = df['stopword_removal'].apply(stem_tokens)

In [66]:
# Cek hasil 
df[["stopword_removal", "stemming_output"]].head()

,stopword_removal,stemming_output
0,"[direktur, jenderal, pajak, suryo, utomo, meng...","[direktur, jenderal, pajak, suryo, utomo, ungk..."
1,"[rata, berada, bawah, detik, performa, terbaik...","[rata, ada, bawah, detik, performa, baik, cata..."
2,"[ditjen, pajak, menilai, performa, coretax, ad...","[ditjen, pajak, nilai, performa, coretax, admi..."
3,"[akhir, marak, penipuan, mengatasnamakan, impl...","[akhir, marak, tipu, mengatasnamakan, implemen..."
4,"[direktur, jenderal, dirjen, pajak, kementeria...","[direktur, jenderal, dirjen, pajak, menteri, u..."


### ✅ **8. Rare Words Removal** → Menghapus kata yang jarang muncul.

In [67]:
word_counts = Counter(word for tokens in df["stemming_output"] for word in tokens)

rare_threshold = 2
rare_words = {word for word, count in word_counts.items() if count < rare_threshold}

def remove_rare_words(tokens):
    return [word for word in tokens if word not in rare_words]

df['rareword_removed'] = df['stemming_output'].apply(remove_rare_words)

In [68]:
# Cek hasil 
df[["stemming_output", "rareword_removed",]].head()

,stemming_output,rareword_removed
0,"[direktur, jenderal, pajak, suryo, utomo, ungk...","[direktur, jenderal, pajak, suryo, utomo, ungk..."
1,"[rata, ada, bawah, detik, performa, baik, cata...","[rata, ada, bawah, detik, performa, baik, cata..."
2,"[ditjen, pajak, nilai, performa, coretax, admi...","[ditjen, pajak, nilai, performa, coretax, admi..."
3,"[akhir, marak, tipu, mengatasnamakan, implemen...","[akhir, marak, tipu, mengatasnamakan, implemen..."
4,"[direktur, jenderal, dirjen, pajak, menteri, u...","[direktur, jenderal, dirjen, pajak, menteri, u..."


### ✅ **9. Delete Empty Rows**

In [69]:
# Cek jumlah baris yang memiliki token kosong setelah preprocessing
df_empty = df[df["rareword_removed"].apply(lambda x: len(x) == 0)]
print(f"Jumlah baris kosong: {len(df_empty)}")

Jumlah baris kosong: 0


## 📌 **Save Data**

In [70]:
# Simpan DataFrame
output_path = "../data/hasil_preprocessing.xlsx"
df.to_excel(output_path, index=False)